In [1]:
import csv
from influxdb_client import InfluxDBClient



In [2]:
# Connection Params for InfluxDB
url = "http://Pi-Ip-address:8086"
token = "API-token"
org = "Your-Org"
bucket = "SensorBucket"

# Time Range for export (will add usr input)
start = "-30m"
stop = "now()"




In [3]:

# List of metrics (measurements and fields) to pull
metrics = [
    {"measurement": "airgradient_co2_ppm"},
    {"measurement": "airgradient_humidity_compensated_percent"},
    {"measurement": "airgradient_humidity_percent"},
    {"measurement": "airgradient_temperature_celsius"},
    {"measurement": "airgradient_co2_ppm"},
    {"measurement": "airgradient_humidity_compensated_percent"},
    {"measurement": "airgradient_humidity_percent"},
    {"measurement": "airgradient_nox_index"},
    {"measurement": "airgradient_nox_raw"},
    {"measurement": "airgradient_pm0d3_p100ml"},
    {"measurement": "airgradient_pm10_ugm3"},
    {"measurement": "airgradient_pm1_ugm3"},
    {"measurement": "airgradient_pm2d5_ugm3"},
    {"measurement": "airgradient_temperature_compensated_celsius"},
    {"measurement": "airgradient_tvoc_index"},
    {"measurement": "airgradient_tvoc_raw"},
]

In [ ]:

# Create the InfluxDB client
client = InfluxDBClient(url=url, token=token, org=org)

# Define a Flux query to retrieve multiple metrics together
query = f'''
from(bucket: "{bucket}")
|> range(start: {start}, stop: {stop})
|> filter(fn: (r) => r._measurement == "{metrics[0]['measurement']}")
'''  # Initial query for the first metric

# Build the query for additional metrics
for metric in metrics[1:]:
    query += f'''
|> join(
    tables: {{main: .}},
    on: ["_time"],
    tables_2: from(bucket: "{bucket}")
    |> range(start: {start}, stop: {stop})
    |> filter(fn: (r) => r._measurement == "{metric['measurement']}")
    )
    |> yield()
'''

# Run the query
query_api = client.query_api()
tables = query_api.query(query)

# Process results into a dictionary with timestamps as keys
data_dict = {}

for table in tables:
    for record in table.records:
        time = record.get_time()
        measurement = record.get_measurement()
        field = record.get_field()
        value = record.get_value()

        # Initialize dictionary entry for the time if it doesn't exist
        if time not in data_dict:
            data_dict[time] = {}
        
        # Store the value for the corresponding field
        data_dict[time][measurement] = value

# Write data to a CSV file
with open('influxdb_export.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    header = ["time"] + [metric['measurement'] for metric in metrics]
    csvwriter.writerow(header)

    # Write data rows
    for time, values in data_dict.items():
        row = [time] + [values.get(metric['measurement'], None) for metric in metrics]
        csvwriter.writerow(row)

print("Data exported to influxdb_export.csv")

# Close the client
client.close()

In [ ]:
QUERY='''from(bucket: "SensorBucket")
  |> range(start: -30m)
  |> filter(fn: (r) => r["_measurement"] == "airgradient_temperature_celsius"
      or r["_measurement"] == "airgradient_co2_ppm"
      or r["_measurement"] == "airgradient_humidity_compensated_percent"
      or r["_measurement"] == "airgradient_humidity_percent"
      or r["_measurement"] == "airgradient_nox_index"
      or r["_measurement"] == "airgradient_nox_raw"
      or r["_measurement"] == "airgradient_pm0d3_p100ml"
      or r["_measurement"] == "airgradient_pm10_ugm3"
      or r["_measurement"] == "airgradient_pm1_ugm3"
      or r["_measurement"] == "airgradient_pm2d5_ugm3"
      or r["_measurement"] == "airgradient_temperature_compensated_celsius"
      or r["_measurement"] == "airgradient_tvoc_index"
      or r["_measurement"] == "airgradient_tvoc_raw")
  |> keep(columns: ["_time", "_value", "_measurement"])
  |> pivot(rowKey:["_time"], columnKey: ["_measurement"], valueColumn: "_value")
'''
client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()
tables = query_api.query(query)

In [14]:

# Create the InfluxDB client
client = InfluxDBClient(url=url, token=token, org=org)

# Define a Flux query to retrieve multiple metrics together
query = """from(bucket: "SensorBucket")
  |> range(start: -30m)
  |> filter(fn: (r) => r["_measurement"] == "airgradient_temperature_celsius"
    or r["_measurement"] == "airgradient_co2_ppm"
    or r["_measurement"] == "airgradient_humidity_compensated_percent"
    or r["_measurement"] == "airgradient_humidity_percent"
    or r["_measurement"] == "airgradient_nox_index"
    or r["_measurement"] == "airgradient_nox_raw"
    or r["_measurement"] == "airgradient_pm0d3_p100ml"
    or r["_measurement"] == "airgradient_pm10_ugm3"
    or r["_measurement"] == "airgradient_pm1_ugm3"
    or r["_measurement"] == "airgradient_pm2d5_ugm3"
    or r["_measurement"] == "airgradient_temperature_compensated_celsius"
    or r["_measurement"] == "airgradient_tvoc_index"
    or r["_measurement"] == "airgradient_tvoc_raw")
  |> filter(fn: (r) => r["_field"] == "gauge")
"""

# Run the query
query_api = client.query_api()
tables = query_api.query(query)


client.close()

  |> keep(columns: ["_time", "_value", "_measurement"])
  |> pivot(rowKey:["_time"], columnKey: ["_measurement"], valueColumn: "_value")

In [15]:
print(tables)

[<FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 10 columns, 179 records>, <FluxTable: 10 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 11 columns, 179 records>, <FluxTable: 10 columns, 179 records>, <FluxTable: 10 columns, 179 records>]


In [18]:

# Process results into a dictionary with timestamps as keys
data_dict = {}

for table in tables:
    for record in table.records:
        # print(record)
        time = record.get_time()
        # print(time)
        value = record.get_value()
        # print(value)
        measurement = record.get_measurement()


        # Initialize dictionary entry for the time if it doesn't exist
        if time not in data_dict:
            data_dict[time] = {}
        
        # Store the value for the corresponding field
        data_dict[time][measurement] = value


In [21]:
with open('influxdb_export.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    header = ["time"] + [metric['measurement'] for metric in metrics]
    csvwriter.writerow(header)

    # Write data rows
    for time, values in data_dict.items():
        row = [time] + [values.get(metric['measurement'], None) for metric in metrics]
        csvwriter.writerow(row)

print("Data exported to influxdb_export.csv")



Data exported to influxdb_export.csv
